### SAS 3

In [104]:
import pandas as pd
import numpy as np 

In [105]:
comp = pd.read_csv(r"C:\Users\User\OneDrive\Desktop/compustat_permno.csv")
sample_data = pd.read_csv(r"C:\Users\User\OneDrive\Desktop/assignment1_sample_data.csv")
crsp = pd.read_csv(r"C:\Users\User\OneDrive\Desktop/CRSP_M.csv")

In [106]:
print(comp.dtypes)
comp.head()

gvkey         int64
datadate      int64
itcb        float64
pstk        float64
pstkl       float64
pstkrv      float64
seq         float64
txdb        float64
permno      float64
permco      float64
dtype: object


,gvkey,datadate,itcb,pstk,pstkl,pstkrv,seq,txdb,permno,permco
0,1000,19611231,0.0,NaN,0.0,NaN,NaN,0.000,NaN,NaN
1,1000,19621231,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN
2,1000,19631231,0.0,0.0,0.0,0.0,0.553,0.008,NaN,NaN
3,1000,19641231,0.0,0.0,0.0,0.0,0.607,0.020,NaN,NaN
4,1000,19651231,0.0,0.0,0.0,0.0,0.491,0.000,NaN,NaN


In [107]:
comp = comp.dropna(subset=['permno']) # drop na permno
print(comp['permno'].isna().sum())
print(comp.shape)

0
(264450, 10)


In [108]:
comp['datadate'] = pd.to_datetime(comp['datadate'], format='%Y%m%d')
comp['year'] = comp['datadate'].dt.year
print(comp.shape)

(264450, 11)


In [109]:
col = ['pstkrv', 'pstkl', 'pstk', 'seq', 'txdb', 'itcb']
comp['pref'] = comp.get("pstkrv", np.nan)
if 'pstkl' in comp.columns:
    comp['pref'] = comp['pref'].fillna(comp['pstkl'])
if 'pstk' in comp.columns:
    comp['pref'] = comp['pref'].fillna(comp['pstk'])


print(comp.shape)

(264450, 12)


In [110]:
# compute BE = SEQ + TXDB + ITCB - Preferred Stock
comp['BE'] = comp['seq'] + comp['txdb'].fillna(0) + comp['itcb'].fillna(0) - comp['pref'].fillna(0)
comp['BE'] = comp['BE'].where(comp['BE'] > 0)

In [111]:
comp = comp.dropna(subset=['BE'])

In [112]:
print(comp['BE'].isna().sum())

0


In [113]:
 # keep the last observation in each group 
idx = comp.groupby(['gvkey', 'permno', 'year'])['datadate'].idxmax()
comp = comp.loc[idx].reset_index(drop=True)
print(comp.shape)

(233752, 13)


### SAS 5 

In [114]:
print(crsp.dtypes)
crsp.head()

DATE        int64
DLSTCD    float64
PERMNO      int64
SHRCD       int64
EXCHCD      int64
SICCD     float64
DLRET     float64
PERMCO      int64
PRC       float64
VOL       float64
RET       float64
SHROUT    float64
ALTPRC    float64
rf        float64
dtype: object


,DATE,DLSTCD,PERMNO,SHRCD,EXCHCD,SICCD,DLRET,PERMCO,PRC,VOL,RET,SHROUT,ALTPRC,rf
0,19610131,NaN,10006,10,1,3740.0,NaN,22156,50.25,939.0,0.322368,1420.0,50.25,0.0019
1,19610131,NaN,10014,10,1,3710.0,NaN,22157,4.00,395.0,0.000000,2504.0,4.00,0.0019
2,19610131,NaN,10030,10,1,3310.0,NaN,22160,41.75,280.0,0.087948,1627.0,41.75,0.0019
3,19610131,NaN,10057,11,1,3540.0,NaN,20020,54.00,152.0,0.142857,500.0,54.00,0.0019
4,19610131,NaN,10102,10,1,2810.0,NaN,22164,79.50,480.0,0.032468,3965.0,79.50,0.0019


In [115]:
crsp[crsp['DLSTCD'].between(551, 573)]

,DATE,DLSTCD,PERMNO,SHRCD,EXCHCD,SICCD,DLRET,PERMCO,PRC,VOL,RET,SHROUT,ALTPRC,rf
12066,19611130,551.0,25187,10,1,7810.0,0.086538,23317,NaN,6.0,NaN,889.0,53.0000,0.0015
21828,19620731,551.0,11076,10,1,4011.0,0.013193,22245,NaN,109.0,NaN,438.0,24.0000,0.0027
21829,19620731,551.0,17945,10,1,6733.0,0.056075,22817,NaN,13.0,NaN,672.0,-27.5000,0.0027
29715,19621130,551.0,33689,10,2,3520.0,0.010000,23943,NaN,3.0,NaN,387.0,12.0000,0.0020
29716,19621130,561.0,19967,10,1,2082.0,-0.120000,22962,NaN,67.0,NaN,1533.0,0.6250,0.0020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2913813,20121031,560.0,91412,11,2,3674.0,-0.738903,51093,NaN,15793.0,NaN,44520.0,0.0600,0.0001
2913814,20121031,561.0,40970,11,2,1311.0,-0.213199,2183,NaN,1256.0,NaN,437.0,1.9700,0.0001
2917504,20121130,552.0,92088,11,3,7389.0,-0.350105,52368,NaN,1694.0,NaN,18300.0,0.3301,0.0001
2917505,20121130,552.0,92779,11,3,7812.0,0.063830,53081,NaN,122.0,NaN,15196.0,0.4000,0.0001


In [116]:
crsp['DATE'] = pd.to_datetime(crsp['DATE'], format='%Y%m%d')

In [117]:
col = ['PRC', 'SHROUT', 'ALTPRC', 'ME']
query = (crsp['ALTPRC'].abs() > 0) & (crsp['SHROUT']> 0)
crsp.loc[query, 'ME'] = crsp['ALTPRC'].abs() * crsp['SHROUT'] / 1000 # calculate ME in million dollars
crsp.head()

# calculating return considering delisiting 
query = crsp['DLRET'].isna() & crsp['DLSTCD'].notna()
query_30 = (
    crsp['DLSTCD'].isin([500, 520, 574, 580, 584]) |
    crsp['DLSTCD'].between(551, 573)
)
crsp.loc[query & query_30, 'DLRET'] = -0.3
crsp.loc[query & ~query_30, 'DLRET'] = -1 

In [118]:
#compute adjusted return considering delisting return
crsp['adj_ret'] = crsp['RET']
query_both = crsp['DLSTCD'].notna() & crsp['DLRET'].notna()& crsp['RET'].notna()
crsp.loc[query_both, 'adj_ret'] = (1 + crsp['RET']) * (1 + crsp['DLRET']) - 1

query_only = crsp['DLSTCD'].notna() & crsp['DLRET'].notna()& crsp['RET'].isna()
crsp.loc[query_only, 'adj_ret'] = crsp['DLRET'] 

In [119]:
crsp['excess_ret'] = crsp['adj_ret'] - crsp['rf']

In [120]:
print(crsp.shape)
crsp.head()

(2921193, 17)


,DATE,DLSTCD,PERMNO,SHRCD,EXCHCD,SICCD,DLRET,PERMCO,PRC,VOL,RET,SHROUT,ALTPRC,rf,ME,adj_ret,excess_ret
0,1961-01-31,NaN,10006,10,1,3740.0,NaN,22156,50.25,939.0,0.322368,1420.0,50.25,0.0019,71.35500,0.322368,0.320468
1,1961-01-31,NaN,10014,10,1,3710.0,NaN,22157,4.00,395.0,0.000000,2504.0,4.00,0.0019,10.01600,0.000000,-0.001900
2,1961-01-31,NaN,10030,10,1,3310.0,NaN,22160,41.75,280.0,0.087948,1627.0,41.75,0.0019,67.92725,0.087948,0.086048
3,1961-01-31,NaN,10057,11,1,3540.0,NaN,20020,54.00,152.0,0.142857,500.0,54.00,0.0019,27.00000,0.142857,0.140957
4,1961-01-31,NaN,10102,10,1,2810.0,NaN,22164,79.50,480.0,0.032468,3965.0,79.50,0.0019,315.21750,0.032468,0.030568


In [121]:
# aggregate ME for a given permco and date
crsp = crsp.dropna(subset=['ME'])
me_agg =crsp.groupby(['PERMCO', 'DATE'])['ME'].sum().reset_index().rename(columns={'ME': 'ME_sum'})

In [122]:
me_agg.head()

,PERMCO,DATE,ME_sum
0,3,1977-01-31,6.412500
1,3,1977-02-28,6.946875
2,3,1977-03-31,6.056250
3,3,1977-04-29,6.501563
4,3,1977-05-31,6.857812


In [123]:
query = crsp.groupby(['PERMCO', 'DATE'])['ME'].idxmax() #sort ME within PERMCO and DATE, get the index of the max ME
crsp_max = crsp.loc[query, ['PERMNO', 'DATE', 'PERMCO']]
crsp_max.head()

,PERMNO,DATE,PERMCO
534930,12160,1977-01-31,3
539767,12160,1977-02-28,3
544586,12160,1977-03-31,3
549404,12160,1977-04-29,3
554226,12160,1977-05-31,3


In [124]:
crsp_agg = crsp_max.merge(me_agg, on =['PERMCO', 'DATE'], how='left') #total market equity for each permco and date
crsp_agg.head()
crsp_agg.shape

(2878883, 4)

In [125]:
crsp = crsp_agg.merge(crsp.drop(columns = ['ME']), on= ['PERMNO', 'DATE', 'PERMCO'], how = 'left').drop_duplicates(subset=['PERMNO', 'DATE']).rename(columns={'ME_sum': 'ME'})
crsp.shape

(2878883, 17)

In [126]:
crsp.head()

,PERMNO,DATE,PERMCO,ME,DLSTCD,SHRCD,EXCHCD,SICCD,DLRET,PRC,VOL,RET,SHROUT,ALTPRC,rf,adj_ret,excess_ret
0,12160,1977-01-31,3,6.412500,NaN,10,3,NaN,NaN,-4.5000,NaN,NaN,1425.0,-4.5000,0.0036,NaN,NaN
1,12160,1977-02-28,3,6.946875,NaN,10,3,NaN,NaN,-4.8750,NaN,0.083333,1425.0,-4.8750,0.0035,0.083333,0.079833
2,12160,1977-03-31,3,6.056250,NaN,10,3,NaN,NaN,-4.2500,NaN,-0.128205,1425.0,-4.2500,0.0038,-0.128205,-0.132005
3,12160,1977-04-29,3,6.501563,NaN,10,3,NaN,NaN,-4.5625,NaN,0.073529,1425.0,-4.5625,0.0038,0.073529,0.069729
4,12160,1977-05-31,3,6.857812,NaN,10,3,NaN,NaN,-4.8125,NaN,0.054795,1425.0,-4.8125,0.0037,0.054795,0.051095


### SAS 6

In [127]:
# adding date related columns
crsp['year'] = crsp['DATE'].dt.year
crsp['month'] = crsp['DATE'].dt.month

In [128]:
# ME_june 

query = (crsp['month'] == 6) &  (crsp['ME'].notna())
me_june = crsp.loc[query, ['PERMNO', 'DATE', 'ME']].rename(columns={'ME': 'ME_june'})
me_june['t'] = me_june['DATE'].dt.year
me_june.drop(columns='DATE', inplace=True)
me_june.head()

,PERMNO,ME_june,t
5,12160,6.857812,1977
17,12160,8.371875,1978
29,12160,11.400000,1979
41,12160,15.682500,1980
53,12160,16.433500,1981


In [129]:
# ME_last_dec 
me_dec = crsp.loc[(crsp['month'] == 12) & (crsp['ME'].notna()), ['PERMNO', 'DATE', 'ME']].rename(columns={'ME': 'ME_last_dec'})
me_dec['t'] = me_dec['DATE'].dt.year + 1 
me_dec.drop(columns='DATE', inplace=True)
me_dec.head()

,PERMNO,ME_last_dec,t
11,12160,6.946875,1978
23,12160,8.817187,1979
35,12160,17.837250,1980
47,12160,19.603125,1981
59,12160,20.899125,1982


In [130]:
# BE_last_year 

be_prev = comp[['permno', 'year', 'BE']].rename(columns={'permno': 'PERMNO', 'BE': 'BE_last_year'})
be_prev['t'] = be_prev['year'] + 1 
be_prev.drop(columns='year', inplace=True)
be_prev.head()

,PERMNO,BE_last_year,t
0,25881.0,10.544,1971
1,25881.0,8.382,1972
2,25881.0,7.309,1973
3,25881.0,8.798,1974
4,25881.0,8.279,1975


In [131]:
# merge all together 

me_bm = me_june.merge(be_prev, on=['PERMNO', 't'], how='inner').merge(me_dec, on=['PERMNO', 't'], how='inner')
me_bm['BM'] = me_bm['BE_last_year'] / me_bm['ME_last_dec'] #ME_last_dec 로 수정 필요 
me_bm = me_bm[['PERMNO', 't', 'ME_june', 'BM']]
me_bm.head()

,PERMNO,t,ME_june,BM
0,12160,1978,8.371875,2.640180
1,12160,1979,11.400000,2.241985
2,12160,1980,15.682500,1.217452
3,12160,1982,23.364000,1.233353
4,63845,1973,77.961625,0.135024


In [132]:
crsp['t'] = crsp['year'].where(crsp['month'] > 6 , crsp['year'] - 1)
crsp.head()

,PERMNO,DATE,PERMCO,ME,DLSTCD,SHRCD,EXCHCD,SICCD,DLRET,PRC,VOL,RET,SHROUT,ALTPRC,rf,adj_ret,excess_ret,year,month,t
0,12160,1977-01-31,3,6.412500,NaN,10,3,NaN,NaN,-4.5000,NaN,NaN,1425.0,-4.5000,0.0036,NaN,NaN,1977,1,1976
1,12160,1977-02-28,3,6.946875,NaN,10,3,NaN,NaN,-4.8750,NaN,0.083333,1425.0,-4.8750,0.0035,0.083333,0.079833,1977,2,1976
2,12160,1977-03-31,3,6.056250,NaN,10,3,NaN,NaN,-4.2500,NaN,-0.128205,1425.0,-4.2500,0.0038,-0.128205,-0.132005,1977,3,1976
3,12160,1977-04-29,3,6.501563,NaN,10,3,NaN,NaN,-4.5625,NaN,0.073529,1425.0,-4.5625,0.0038,0.073529,0.069729,1977,4,1976
4,12160,1977-05-31,3,6.857812,NaN,10,3,NaN,NaN,-4.8125,NaN,0.054795,1425.0,-4.8125,0.0037,0.054795,0.051095,1977,5,1976


In [133]:
ret_me_bm = crsp.merge(me_bm, on=['PERMNO', 't'], how='inner')
ret_me_bm.shape

(2196265, 22)

In [134]:
ret_me_bm = ret_me_bm.sort_values(by=['PERMNO', 'DATE']).reset_index(drop=True)
ret_me_bm.head()

,PERMNO,DATE,PERMCO,ME,DLSTCD,SHRCD,EXCHCD,SICCD,DLRET,PRC,...,SHROUT,ALTPRC,rf,adj_ret,excess_ret,year,month,t,ME_june,BM
0,10001,1987-07-31,7953,5.9460,NaN,11,3,4920.0,NaN,6.0000,...,991.0,6.0000,0.0046,0.021277,0.016677,1987,7,1987,5.822125,1.014415
1,10001,1987-08-31,7953,6.4415,NaN,11,3,4920.0,NaN,6.5000,...,991.0,6.5000,0.0047,0.083333,0.078633,1987,8,1987,5.822125,1.014415
2,10001,1987-09-30,7953,6.2000,NaN,11,3,4920.0,NaN,6.2500,...,992.0,6.2500,0.0045,-0.022308,-0.026808,1987,9,1987,5.822125,1.014415
3,10001,1987-10-30,7953,6.3240,NaN,11,3,4920.0,NaN,6.3750,...,992.0,6.3750,0.0060,0.020000,0.014000,1987,10,1987,5.822125,1.014415
4,10001,1987-11-30,7953,6.1380,NaN,11,3,4920.0,NaN,6.1875,...,992.0,6.1875,0.0035,-0.029412,-0.032912,1987,11,1987,5.822125,1.014415


In [135]:
ret_me_bm.columns

Index(['PERMNO', 'DATE', 'PERMCO', 'ME', 'DLSTCD', 'SHRCD', 'EXCHCD', 'SICCD',
       'DLRET', 'PRC', 'VOL', 'RET', 'SHROUT', 'ALTPRC', 'rf', 'adj_ret',
       'excess_ret', 'year', 'month', 't', 'ME_june', 'BM'],
      dtype='object')

In [136]:
ret_me_bm['altprc_lag1'] = ret_me_bm.groupby('PERMNO')['ALTPRC'].shift(1)
ret_me_bm['ME_lag1'] = ret_me_bm.groupby('PERMNO')['ME'].shift(1)
ret_me_bm['prev_date'] = ret_me_bm.groupby('PERMNO')['DATE'].shift(1)


gap_month = (ret_me_bm['DATE'].dt.year - ret_me_bm['prev_date'].dt.year) * 12 + (ret_me_bm['DATE'].dt.month - ret_me_bm['prev_date'].dt.month)
query = (ret_me_bm['PERMNO'] != ret_me_bm.groupby('PERMNO')['PERMNO'].shift(1)) | (gap_month > 1) | (gap_month.isna())
ret_me_bm.loc[query, ['altprc_lag1', 'ME_lag1']] = np.nan


In [137]:
query = ret_me_bm[['adj_ret', 'ME_lag1', 'ME_june', 'BM']].notna().all(axis=1)
assignment_data = ret_me_bm.loc[query, ['PERMNO', 'DATE', 'year', 'EXCHCD', 'SICCD', 'adj_ret', 'excess_ret', 'altprc_lag1', 'ME_lag1', 'ME_june', 'BM' ]].sort_values(by=['PERMNO', 'DATE']).reset_index(drop=True)

print(assignment_data.shape)
assignment_data.head()

assignment_agg = assignment_data.copy()

(2173794, 11)


In [144]:
assignment_data.head(25)

,PERMNO,DATE,year,EXCHCD,SICCD,adj_ret,excess_ret,altprc_lag1,ME_lag1,ME_june,BM
0,10001,1987-08-31,1987,3,4920.0,0.083333,0.078633,6.0000,5.94600,5.822125,1.014415
1,10001,1987-09-30,1987,3,4920.0,-0.022308,-0.026808,6.5000,6.44150,5.822125,1.014415
2,10001,1987-10-30,1987,3,4920.0,0.020000,0.014000,6.2500,6.20000,5.822125,1.014415
3,10001,1987-11-30,1987,3,4920.0,-0.029412,-0.032912,6.3750,6.32400,5.822125,1.014415
4,10001,1987-12-31,1987,3,4920.0,-0.033535,-0.037435,6.1875,6.13800,5.822125,1.014415
5,10001,1988-01-29,1988,3,4920.0,0.063830,0.060930,5.8750,5.82800,5.822125,1.014415
6,10001,1988-02-29,1988,3,4920.0,0.080000,0.075400,6.2500,6.20000,5.822125,1.014415
7,10001,1988-03-31,1988,3,4920.0,-0.076296,-0.080696,6.7500,6.69600,5.822125,1.014415
8,10001,1988-04-29,1988,3,4920.0,0.030612,0.026012,6.1250,6.07600,5.822125,1.014415
9,10001,1988-05-31,1988,3,4920.0,0.019802,0.014702,-6.3125,6.26200,5.822125,1.014415


In [138]:
assignment_agg['yyyymm'] = assignment_agg['DATE'].dt.strftime('%Y%m')
selected_year = ['197012', '198012','199012','200012','201012']
query = assignment_agg['yyyymm'].isin(selected_year)

In [139]:
assignment_agg.loc[query, ].groupby('yyyymm').agg(
    mean = ('excess_ret', 'mean'),
    std = ('excess_ret', 'std'),
    min = ('excess_ret', 'min'),
    max = ('excess_ret', 'max'), 
    N_permnos = ('PERMNO', 'nunique')
).reset_index()

,yyyymm,mean,std,min,max,N_permnos
0,197012,0.080410,0.110435,-0.304200,0.707338,1701
1,198012,-0.045508,0.124436,-1.013100,2.272614,3681
2,199012,-0.001734,0.182765,-1.006000,1.994000,4386
3,200012,-0.013670,0.245187,-0.999667,3.245000,5474
4,201012,0.082710,0.142284,-0.844100,2.400847,3574


### compare to sample data

In [140]:
sample_data.describe()

,permno,date,year,exchcd,siccd,retadj,eretadj,altprc_lag1,ME_lag1,ME_Jun,BM
count,3198.000000,3198.0,3198.0,3198.000000,3198.000000,3198.000000,3198.000000,3198.000000,3198.000000,3198.000000,3198.000000
mean,68969.367417,20121231.0,2012.0,2.227955,5035.565041,0.026761,0.026661,66.307163,4348.820794,4217.002132,0.923025
std,27968.735150,0.0,0.0,0.940657,2181.075090,0.117142,0.117142,2332.963722,20326.911889,20087.698277,0.924562
min,10001.000000,20121231.0,2012.0,1.000000,170.000000,-0.774834,-0.774934,-23.654999,0.888375,0.854465,0.000795
25%,53503.750000,20121231.0,2012.0,1.000000,3545.000000,-0.015974,-0.016074,5.200000,103.046618,102.681551,0.381303
50%,82242.500000,20121231.0,2012.0,3.000000,4924.000000,0.021051,0.020951,14.410000,464.349117,446.737531,0.705639
75%,89695.750000,20121231.0,2012.0,3.000000,6330.000000,0.065244,0.065144,32.355000,1959.077633,1891.924352,1.132053
max,93436.000000,20121231.0,2012.0,3.000000,9999.000000,2.520000,2.519900,131916.000000,550568.241320,547363.344000,10.565308


In [141]:
sample_data['date'] = pd.to_datetime(sample_data['date'], format='%Y%m%d')
sample_data['yyyymm'] = sample_data['date'].dt.strftime('%Y%m')
sample_data.groupby('yyyymm').agg(
    mean = ('eretadj', 'mean'),
    std = ('eretadj', 'std'),
    min = ('eretadj', 'min'),
    max = ('eretadj', 'max'), 
    N_permnos = ('permno', 'nunique')
)

,mean,std,min,max,N_permnos
yyyymm,,,,,
201212,0.026661,0.117142,-0.774934,2.5199,3198


In [142]:
query = assignment_agg['yyyymm'] == '201212'
assignment_agg.loc[query, ].groupby('yyyymm').agg(
    mean = ('excess_ret', 'mean'),
    std = ('excess_ret', 'std'),
    min = ('excess_ret', 'min'),
    max = ('excess_ret', 'max'), 
    N_permnos = ('PERMNO', 'nunique')
).reset_index()

,yyyymm,mean,std,min,max,N_permnos
0,201212,0.027057,0.115648,-0.774934,2.5199,3365
